# Подключение библиотек

In [1]:
# подключение библиотеки pandas
import pandas as pd
import numpy as np
# подключение библиотеки matplotlib.pyplot
import matplotlib.pyplot as plt
# подключение библиотеки seaborn
import seaborn as sns
# подключение модулей из библиотеки sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

# Читанием исходный DataFrame (df) из файла

In [2]:
df = pd.read_csv("data/unclean_smartwatch_health_data.csv")

# Подготовка данных, группировка и избавление от пропусков

## Проводим группировку в столбце Activity Level

### Функция для нормализации значений в столбце Activity Level

In [3]:
def activity_level_group(x):
  if x in ['Highly_Active', 'Highly Active']:    # исправляем опечатки
    return 'Highly Active'                       # и
  if x in ['Seddentary', 'Sedentary']:           # приводим  к общему 
    return 'Sedentary'                           # виду значения в столбце
  if x in ['Actve', 'Active']:
    return 'Active'
  return x

### Вызываем функцию для нормализации значений и выводим новые уникальные значения

In [4]:
df['Activity Level'] = df['Activity Level'].apply(activity_level_group)  # вызов функции
df['Activity Level'].value_counts(dropna=False)

Activity Level
Sedentary        3333
Active           3265
Highly Active    3202
NaN               200
Name: count, dtype: int64

## Просмотр типов данных по каждому столбцу

In [5]:
df.dtypes

User ID                   float64
Heart Rate (BPM)          float64
Blood Oxygen Level (%)    float64
Step Count                float64
Sleep Duration (hours)     object
Activity Level             object
Stress Level               object
dtype: object

## Просмотр сводной статистики всех числовых столбцов

In [6]:
df.describe()

,User ID,Heart Rate (BPM),Blood Oxygen Level (%),Step Count
count,9799.000000,9600.000000,9700.000000,9900.000000
mean,3007.480253,76.035462,97.841581,6985.685885
std,1150.581542,19.412483,1.732863,6885.809680
min,1001.000000,40.000000,90.791208,0.910138
25%,1997.500000,64.890152,96.662683,2021.039657
50%,2998.000000,75.220601,98.010642,4962.534599
75%,4004.000000,85.198249,99.376179,9724.902880
max,4999.000000,296.593970,100.000000,62486.690753


## Убираем отсутствующих значения

### Просмотр количества отсутсвующих значений по всем столбцам

In [7]:
df.isnull().sum()

User ID                   201
Heart Rate (BPM)          400
Blood Oxygen Level (%)    300
Step Count                100
Sleep Duration (hours)    150
Activity Level            200
Stress Level              200
dtype: int64

### Заменяем все пропуски в столбце User ID случайными числами

In [8]:
df["User ID"] = df['User ID'].transform(lambda x: np.random.rand())
df.isnull().sum() # просмотр количества отсутствующих значений по столбцам

User ID                     0
Heart Rate (BPM)          400
Blood Oxygen Level (%)    300
Step Count                100
Sleep Duration (hours)    150
Activity Level            200
Stress Level              200
dtype: int64

### Заменяем пропущенные значения в остальных столбцах средними значениями, рассчитанными для соответствующих групп

#### Столбец Heart Rate (BPM)

In [9]:
# разделяем DataFrame на группы на основе значений в столбцах 'Activity Level', 'Stress Level'
# и 'Sleep Duration (hours)'
df['Heart Rate (BPM)'] = df.groupby(['Activity Level','Stress Level'],
                                     # заменяем пропуски средним значением для каждой группы
                                          dropna=False)['Heart Rate (BPM)'].transform(lambda x: x.fillna(x.mean()))
df.isnull().sum()  # просмотр количества отсуктствующих значений по столбцам

User ID                     0
Heart Rate (BPM)            0
Blood Oxygen Level (%)    300
Step Count                100
Sleep Duration (hours)    150
Activity Level            200
Stress Level              200
dtype: int64

#### Столбец Blood Oxygen Level (%)

In [10]:
# разделяем DataFrame на группы на основе значений в столбце 'Activity Level'
df['Blood Oxygen Level (%)'] = df.groupby(['Activity Level'],
                                     # заменяем пропуски средним значением для каждой группы
                                          dropna=False)['Blood Oxygen Level (%)'].transform(lambda x: x.fillna(x.mean()))
df.isnull().sum()  # просмотр количества отсуктствующих значений по столбцам

User ID                     0
Heart Rate (BPM)            0
Blood Oxygen Level (%)      0
Step Count                100
Sleep Duration (hours)    150
Activity Level            200
Stress Level              200
dtype: int64

#### Столбец Step Count

In [11]:
# разделяем DataFrame на группы на основе значений в столбце 'Activity Level'
df['Step Count'] = df.groupby(['Activity Level'],
                                     # заменяем пропуски средним значением для каждой группы
                                          dropna=False)['Step Count'].transform(lambda x: x.fillna(x.mean()))
df.isnull().sum()  # просмотр количества отсуктствующих значений по столбцам

User ID                     0
Heart Rate (BPM)            0
Blood Oxygen Level (%)      0
Step Count                  0
Sleep Duration (hours)    150
Activity Level            200
Stress Level              200
dtype: int64

#### Просмотр доли пропусков по каждому столбцу

In [12]:
df.isna().mean().sort_values(ascending=False)

Stress Level              0.020
Activity Level            0.020
Sleep Duration (hours)    0.015
Blood Oxygen Level (%)    0.000
Heart Rate (BPM)          0.000
User ID                   0.000
Step Count                0.000
dtype: float64

#### Столбец Sleep Duration (hours)

##### Просмотр типа данных в столбце

In [13]:
df['Sleep Duration (hours)'].dtypes

dtype('O')

##### Просмотр уникальных значений в столбце

In [14]:
df['Sleep Duration (hours)'].value_counts(dropna=False)

Sleep Duration (hours)
ERROR                 247
NaN                   150
8.021722201567524       1
5.814348465914493       1
5.691001039740254       1
                     ... 
7.144240087818028       1
7.462106906129147       1
4.546930592918312       1
3.9578798403336597      1
8.531278990913485       1
Name: count, Length: 9605, dtype: int64

##### Преобразование столбца в числовой формат

In [15]:
df['Sleep Duration (hours)'] = pd.to_numeric(df['Sleep Duration (hours)'], downcast="float", errors='coerce') 

##### Просмотр типов данных по столбцам

In [16]:
df.dtypes

User ID                   float64
Heart Rate (BPM)          float64
Blood Oxygen Level (%)    float64
Step Count                float64
Sleep Duration (hours)    float32
Activity Level             object
Stress Level               object
dtype: object

##### Просмотр уникальных значений в столбце

In [17]:
df['Sleep Duration (hours)'].value_counts(dropna=False)

Sleep Duration (hours)
NaN         397
5.531864      2
7.596694      2
5.691001      1
6.276536      1
           ... 
7.144240      1
7.462107      1
4.546931      1
3.957880      1
8.021722      1
Name: count, Length: 9602, dtype: int64

##### Заменяем пропущенные значения в столбце средними значениями, рассчитанными для соответствующих групп

In [18]:
# разделяем DataFrame на группы на основе значений в столбцах 'Activity Level' и 'Stress Level'
df['Sleep Duration (hours)'] = df.groupby(['Activity Level','Stress Level'],
                                          dropna=False)['Sleep Duration (hours)'].transform(lambda x: x.fillna(x.mean()))
df.isnull().sum()   # просмотр количества отсуктствующих значений по столбцам

User ID                     0
Heart Rate (BPM)            0
Blood Oxygen Level (%)      0
Step Count                  0
Sleep Duration (hours)      0
Activity Level            200
Stress Level              200
dtype: int64

#### Просмотр размера DataFrame(df)

In [19]:
len(df)

10000

#### Удаляем отсутствующие значения в остальных столбцах

In [21]:
df = df.dropna()

#### Просмотр новой длины DataFrame(df)

In [22]:
len(df)

9603